In [4]:
#支持的语言
supported_languages=['c',
 'cpp',
 'c_sharp',
 'go',
 'java',
 'php',
 'python',
 'ruby',
 'rust',
 'tsx',
 'fortran',
 'kotlin',
 'cuda',
 'scala']
#该字典为不同语言的主体节点
dict_codebody={'c': ['function_definition', 'struct_specifier'],
 'cpp': ['function_definition', 'namespace_definition', 'class_specifier', 'template_declaration', 'struct_specifier'], 
 'c_sharp': ['namespace_definition', 'class_declaration', 'interface_declaration', 'struct_declaration'], 
 'go': ['type_declaration', 'method_declaration', 'function_declaration'], 
 'java': ['class_declaration', 'interface_declaration'], 
 'php': ['class_declaration', 'expression_statement', 'interface_declaration', 'function_definition'], 
 'python': ['function_definition', 'class_definition'], 'ruby': ['class', 'module', 'method'],
  'rust': ['trait_item', 'struct_item', 'impl_item', 'enum_item', 'function_item', 'type_item', 'mod_item'],
   'tsx': ['lexical_declaration', 'export_statement', 'expression_statement', 'function_declaration', 'class_declaration', 'interface_declaration'], 
   'fortran': ['module', 'subroutine', 'function', 'submodule', 'program'], 
   'kotlin': ['call_expression', 'function_declaration', 'prefix_expression', 'class_declaration', 'object_declaration'], 
   'cuda': ['function_definition', 'template_declaration', 'namespace_definition', 'struct_specifier', 'class_specifier', 'type_definition'], 
   'scala': ['class_definition', 'object_definition', 'function_definition', 'trait_definition']}

#该字典为不同语言主体的前缀，需要在body的前面作为提示信息
'''
dict_codehint={'c': ['comment', ' type_definition', 'declaration'], 
'cpp': ['comment', 'type_definition', 'declaration'], 'c_sharp': ['comment', 'global_statement'], 
'go': ['comment', 'var_declaration'], 'java': ['block_comment', 'line_comment'], 
'php': ['php_tag', 'namespace_definition', 'namespace_use_declaration', 'comment'],
 'python': ['comment', 'expression_statement-string','expression_statement'], 'ruby': ['comment'], 
 'rust': ['line_comment', 'block_comment'], 'tsx': ['comment'], 'fortran': ['comment'], 
 'kotlin': ['comment'], 'cuda': ['comment'], 'scala': ['comment']}'''
dict_codehint={'c': ['comment'], 
'cpp': ['comment'], 'c_sharp': ['comment'], 
'go': ['comment'], 'java': ['block_comment', 'line_comment'], 
'php': [ 'comment'],
 'python': ['comment''expression_statement'], 'ruby': ['comment'], 
 'rust': ['line_comment', 'block_comment'], 'tsx': ['comment'], 'fortran': ['comment'], 
 'kotlin': ['comment'], 'cuda': ['comment'], 'scala': ['comment']}

#映射字典
dict_codegeex2ts={
 'c': 'c',
 'c++': 'cpp',
 'cpp': 'cpp',
 'c#': 'c_sharp',
 'csharp': 'c_sharp',
 'c-sharp':'c_sharp',
 'css': 'css',
 'cuda': 'cuda',
 'dart': 'dart',
 'lua': 'lua',
 'objectivec': 'objc',
 'objective-c': 'objc',
 'python': 'python',
 'perl': 'perl',
 'prolog': 'prolog',
 'lisp': 'elisp',
 'java': 'java',
 'scala': 'scala',
 'tex': 'latex',
 'html': 'html',
 'php': 'php',
 'js': 'javascript',
 'javascript': 'javascript',
 'typescript': 'tsx',
 'go': 'go',
 'rust': 'rust',
 'sql': 'sqlite',
 'kotlin': 'kotlin',
 'ruby': 'ruby',
 'pascal': 'pascal',
 'r': 'r',
 'fortran': 'fortran',
 'matlab': 'matlab'}

In [5]:
from tree_sitter import Language, Parser
#解析代码获取parsetree
def getast_tree(language,text):
    LANGUAGE=Language('build/my-languages.so', language)
    parser=Parser()
    parser.set_language(LANGUAGE)
    tree=parser.parse(bytes(text,"utf8"))
    return tree
def search(node,language):
    nodes=set()
    node_childs = node.children
    if node_childs==[]:
        return set()
    for i in node_childs:
        nodes.add(i.type)
        nodes=nodes|search(i,language)
    return nodes
def test_text(language,text):
    tree=getast_tree(language,text)
    nodes=set()
    nodes=search(tree.root_node,language) 
    for i in nodes:
        if 'comment' in i:
            return True
    return False  
#对根节点的第一层节点进行搜索，将class、function节点加入
def dfs(node,language):
        node_childs = node.children
        if node_childs == []: return
        method_list=[]
        mod=""
        for i in range(len(node_childs)-1,-1,-1):
            if node_childs[i].type in dict_codebody[language]:
                method_list.append(node_childs[i])
                mod='body'
            elif node_childs[i].type in dict_codehint[language]:
                if mod=='body':
                    method_list.append(node_childs[i])
            else:
                mod='trash'
        return method_list
def split_code(language,code):
    "返回所有函数和函数节点信息"
    tree=getast_tree(language,code)
    method_nodes=[]
    method_nodes=dfs(tree.root_node,language)
    code_list = code.split('\n')
    methods_list = []
    method=""
    for n in range(len(method_nodes)-1,-1,-1):
        if method_nodes[n].type in dict_codehint[language]:
            method+=method_nodes[n].text.decode('utf-8') # bytes转str
            method+='\n'
        else:
            method+= method_nodes[n].text.decode('utf-8')
            if test_text(language,method):
                methods_list.append(method)
            method=""
    return methods_list

In [6]:
import json
import json
import os

with open('./test.txt','r') as f:
    text=f.read()
#分割代码
methods_list=[]
method_list=split_code(dict_codegeex2ts['go'],text)
#写文件
for i in method_list:
    print(i)
    print('\n\n\n\n\n')

// Storage represents an interface to almost any database or storage system
type Storage interface {
	Init() error
	Close() error
	ListOfOrgs() ([]types.OrgID, error)
	ListOfClustersForOrg(
		orgID types.OrgID, timeLimit time.Time) ([]types.ClusterName, error,
	)
	ListOfClustersForOrgSpecificRule(
		orgID types.OrgID, ruleID types.RuleSelector, activeClusters []string,
	) ([]ctypes.HittingClustersData, error)
	ReadReportForCluster(
		orgID types.OrgID, clusterName types.ClusterName) (
		[]types.RuleOnReport, types.Timestamp, types.Timestamp, error,
	)
	ReadReportsForClusters(
		clusterNames []types.ClusterName) (map[types.ClusterName]types.ClusterReport, error)
	ReadOrgIDsForClusters(
		clusterNames []types.ClusterName) ([]types.OrgID, error)
	ReadSingleRuleTemplateData(
		orgID types.OrgID, clusterName types.ClusterName, ruleID types.RuleID, errorKey types.ErrorKey,
	) (interface{}, error)
	ReadReportForClusterByClusterName(clusterName types.ClusterName) ([]types.RuleOnReport, types.T